In [1]:
import numpy as np
import random
import pickle
import json
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re 
import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings('ignore')

In [4]:
lb = pickle.load(open('assets/label_encoder.pickle', 'rb'))
tokenizer = pickle.load(open('assets/tokenizer.pickle', 'rb'))
model = load_model('models/model.h5')
emo_res = json.loads(open('data/emo_res.json').read())

In [5]:
lb_chat = pickle.load(open('assets/label_encoder_chat.pickle', 'rb'))
token_chat = pickle.load(open('assets/tokenizer_chat.pickle', 'rb'))
model_chat = load_model('models/model_chat.h5')
int_data = json.loads(open('data/intents.json').read())

In [6]:
def int_clean(sent):
    lemmatizer = WordNetLemmatizer()
    corpus = []
    text = re.sub('[^a-zA-Z]', ' ', sent)  # punctuation removal
    text = text.lower()  # convert to lowercase
    words = text.split()
    words = [lemmatizer.lemmatize(w) for w in words] # lemmatizing
    text = " ".join(words)
    corpus.append(text)
    seq = token_chat.texts_to_sequences(corpus)
    seq_pad = pad_sequences(seq,maxlen=8)
    return seq_pad

In [7]:
def clean(sent):
    lemmatizer = WordNetLemmatizer()
    corpus = []
    text = re.sub('[^a-zA-Z]', ' ', sent) # punctuation removal
    text = text.lower() # convert to lowercase
    words = text.split()
    words = [w for w in words if not w in stop_words] # removing stopwords
    # lemmatizing
    words = [lemmatizer.lemmatize(w) for w in words]
    text = " ".join(words)
    corpus.append(text)
    seq = tokenizer.texts_to_sequences(corpus)
    seq_pad = pad_sequences(seq,maxlen=35)
    return seq_pad

In [8]:
def predict_int(sent):
    sent = int_clean(sent)
    result = lb_chat.inverse_transform(np.argmax(model_chat.predict(sent),axis=1))[0]
    prob =  np.max(model_chat.predict(sent))
    #print(f"{result}: {prob}\n\n")
    return result, prob

In [9]:
def predict(sent):
    sent = clean(sent)
    result = lb.inverse_transform(np.argmax(model.predict(sent),axis=1))[0]
    prob =  np.max(model.predict(sent))
    #print(f"{result}: {prob}\n\n")
    return result, prob

In [10]:
predict_int(str(input('Enter a sentence: ')))

Enter a sentence: bye


2022-08-06 18:42:27.897990: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


('goodbye', 0.73079365)

In [11]:
predict(str(input('Enter a sentence: ')))

Enter a sentence: bye


('sadness', 0.666806)

In [12]:
def response(sent):
    result,prob = predict_int(sent)
    result2,prob2 = predict(sent)
    list_of_ints = int_data['intents']
    if prob > 0.75:
        for i in list_of_ints:
            if i['tag'] == result:
                response_result = random.choice(i['responses'])
                return response_result
    if prob <= 0.75:
        if prob2 > 0.75:
            for i in emo_res:
                if i['emotion'] == result2:
                    response_result = random.choice(i['responses'])
                    return response_result
        else:
            response_result = "Sorry, I don't understand, please ask me something else. You can still choose to listen to these songs."
            return response_result

In [15]:
response('hello')

'Hi! How can I help?'

In [16]:
import requests

In [17]:
api_key = 'ba7998f08e0326c7b05b501610f442e1'
user = 'maggiewang027'
def lastfm_get(payload):
    # define headers and url from the api
    headers = {'user-agent': user}
    url = 'https://ws.audioscrobbler.com/2.0/'
    # add initial setup
    payload['api_key'] = api_key
    payload['format'] = 'json'
    payload['limit'] = 5
    # get the api request
    response = requests.get(url, headers=headers, params=payload)
    return response

In [18]:
lastfm_get({'method': 'tag.getTopTracks','tag':'surprise'}).json()

{'tracks': {'track': [{'name': 'Body Say',
    'duration': '0',
    'mbid': '',
    'url': 'https://www.last.fm/music/Demi+Lovato/_/Body+Say',
    'streamable': {'#text': '0', 'fulltrack': '0'},
    'artist': {'name': 'Demi Lovato',
     'mbid': 'faf4cefb-036c-4c88-b93a-5b03dd0a0e6b',
     'url': 'https://www.last.fm/music/Demi+Lovato'},
    'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'small'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'medium'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'large'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'extralarge'}],
    '@attr': {'rank': '1'}},
   {'name': 'With Strangers',
    'duration': '171',
    'mbid': 'f19dd485-2a5d-45d5-9613-7318f1edf12d',
    'url': 'https://w

In [19]:
def song_get(emo):
    # get the songs based on the emotion detected
    r = lastfm_get({'method': 'tag.getTopTracks','tag':emo})
    payload = r.json()
    l=[]
    artist = payload['tracks']['track'][0]['artist']['name']
    for i in range(0,5):
        song = payload['tracks']['track'][i]['name']
        url = payload['tracks']['track'][i]['url']
        l.append([song,url])
    return l,artist

In [20]:
def similar_get(artist,song):
    r = lastfm_get({'method': 'track.getSimilar','artist':artist,'track':song})
    payload = r.json()
    l=[]
    for i in range(0,5):
        similar_song = payload['similartracks']['track'][i]['name']
        url = payload['similartracks']['track'][i]['url']
        l.append([similar_song,url])
    return l

In [21]:
def recommend(emo):
    #result,prob = predict(sent)
    # get songs and url based on the emotion
    song,artist = song_get(emo)
    # get similar songs and url based on the first song's name and artist
    songs = similar_get(artist,song[0])
    for i in songs:
        song.append(i)
    return song

In [72]:
recommend('longing')

[['Falling', 'https://www.last.fm/music/Harry+Styles/_/Falling'],
 ['Glimpse of Us', 'https://www.last.fm/music/Joji/_/Glimpse+of+Us'],
 ['Death with Dignity',
  'https://www.last.fm/music/Sufjan+Stevens/_/Death+with+Dignity'],
 ['Visions of Gideon',
  'https://www.last.fm/music/Sufjan+Stevens/_/Visions+of+Gideon'],
 ["The Very First Night (Taylor's Version) (From the Vault)",
  'https://www.last.fm/music/Taylor+Swift/_/The+Very+First+Night+(Taylor%27s+Version)+(From+the+Vault)'],
 ['Lights Up', 'https://www.last.fm/music/Harry+Styles/_/Lights+Up'],
 ['Fine Line', 'https://www.last.fm/music/Harry+Styles/_/Fine+Line'],
 ['Heather', 'https://www.last.fm/music/Conan+Gray/_/Heather'],
 ['happier', 'https://www.last.fm/music/Olivia+Rodrigo/_/happier'],
 ['favorite crime',
  'https://www.last.fm/music/Olivia+Rodrigo/_/favorite+crime']]

In [17]:
sadness = ['sadness','depression','suffering','disappointment','shame','sympathy']
anger = ['anger','rage','disgust','envy','torment','frustration']
fear = ['fear','horror','fright','shock','panic','worry']
love = ['love','affection','liking','longing','compassion','passion']
surprise = ['surprise','amazement','astonishment']
joy = ['joy','thrill','contentment','optimism','pride','rapture']

In [18]:
def song_get(emo):
    # get the songs based on the emotion detected
    if emo == 'sadness':
        emotion = random.choice(sadness)
    if emo == 'anger':
        emotion = random.choice(anger)
    if emo == 'fear':
        emotion = random.choice(fear)
    if emo == 'love':
        emotion = random.choice(love)
    if emo == 'surprise':
        emotion = random.choice(surprise)
    if emo == 'joy':
        emotion = random.choice(joy)
    r = lastfm_get({'method': 'tag.getTopTracks','tag':emotion})
    payload = r.json()
    l=[]
    artist = payload['tracks']['track'][0]['artist']['name']
    for i in range(0,5):
        song = payload['tracks']['track'][i]['name']
        url = payload['tracks']['track'][i]['url']
        l.append([song,url])
    return l,artist

In [19]:
l,artist  = song_get('joy')
l

[['High Hopes',
  'https://www.last.fm/music/Panic%21+at+the+Disco/_/High+Hopes'],
 ['Escapee', 'https://www.last.fm/music/Architecture+in+Helsinki/_/Escapee'],
 ["Payne's Bay", 'https://www.last.fm/music/Beirut/_/Payne%27s+Bay'],
 ['That Green Gentleman (Things Have Changed)',
  'https://www.last.fm/music/Panic%21+at+the+Disco/_/That+Green+Gentleman+(Things+Have+Changed)'],
 ['The Peacock', 'https://www.last.fm/music/Beirut/_/The+Peacock']]

In [20]:
def similar_get(artist,song):
    r = lastfm_get({'method': 'track.getSimilar','artist':artist,'track':song})
    payload = r.json()
    l=[]
    for i in range(0,5):
        similar_song = payload['similartracks']['track'][i]['name']
        url = payload['similartracks']['track'][i]['url']
        l.append([similar_song,url])
    return l

In [21]:
l = similar_get('Demi Lovato','Body Say')

In [22]:
l

[['Lonely', 'https://www.last.fm/music/Demi+Lovato/_/Lonely'],
 ['Warrior', 'https://www.last.fm/music/Demi+Lovato/_/Warrior'],
 ['Slumber Party feat. Tinashe',
  'https://www.last.fm/music/Britney+Spears/_/Slumber+Party+feat.+Tinashe'],
 ['Your Body', 'https://www.last.fm/music/Christina+Aguilera/_/Your+Body'],
 ['Me & My Girls', 'https://www.last.fm/music/Selena+Gomez/_/Me+&+My+Girls']]

In [23]:
def recommend(sent):
    result,prob = predict(sent)
    # get songs and url based on the emotion
    song,artist = song_get(result)
    # get similar songs and url based on the first song's name and artist
    songs = similar_get(artist,song[0])
    for i in songs:
        song.append(i)
    return song

In [24]:
recommend('hi')

[['High Hopes',
  'https://www.last.fm/music/Panic%21+at+the+Disco/_/High+Hopes'],
 ['Escapee', 'https://www.last.fm/music/Architecture+in+Helsinki/_/Escapee'],
 ["Payne's Bay", 'https://www.last.fm/music/Beirut/_/Payne%27s+Bay'],
 ['That Green Gentleman (Things Have Changed)',
  'https://www.last.fm/music/Panic%21+at+the+Disco/_/That+Green+Gentleman+(Things+Have+Changed)'],
 ['The Peacock', 'https://www.last.fm/music/Beirut/_/The+Peacock'],
 ['Hey Look Ma, I Made It',
  'https://www.last.fm/music/Panic%21+at+the+Disco/_/Hey+Look+Ma,+I+Made+It'],
 ['This Is Gospel',
  'https://www.last.fm/music/Panic%21+at+the+Disco/_/This+Is+Gospel'],
 ['Radioactive', 'https://www.last.fm/music/Imagine+Dragons/_/Radioactive'],
 ['Pompeii', 'https://www.last.fm/music/Bastille/_/Pompeii'],
 ['whAtEveR it tAkes',
  'https://www.last.fm/music/Imagine+Dragons/_/whAtEveR+it+tAkes']]